In [1]:
# GPU setting
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
# modules setting
from sklearn.metrics import confusion_matrix
import itertools
import os
import numpy as np
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import random
import cv2

/home/project/.pyenv/versions/3.6.3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# AUGMENTED DATA DIR

In [3]:
# cls = ['negative', 'hemorrhagic', 'depressed', 'protruded']
# lesion = {'negative' : ['negative'], 
#           'hemorrhagic': ['red_spot', 'angioectasia', 'active_bleeding'],
#           'depressed': ['erosion', 'ulcer', 'stricture'],
#           'protruded': ['ampulla_of_vater', 'lymphoid_follicles', 'small_bowel_tumor']}
cls = ['negative', 'hemorrhagic']
lesion = {'negative' : ['negative'], 
          'hemorrhagic': ['red_spot', 'angioectasia', 'active_bleeding']}

In [4]:
# path of data folder
dir_data = '/mnt/disk1/project/public/SMhospital/Database/1_ongoing/sm_core/total/'
path_folder = {}

for i in cls:
    path_folder[i] = {}
    for j in lesion[i]:
        temp = dir_data + '{0}/{1}'.format(i, j)
        path_folder[i][j] = temp
        
print(path_folder)

{'negative': {'negative': '/mnt/disk1/project/public/SMhospital/Database/1_ongoing/sm_core/total/negative/negative'}, 'hemorrhagic': {'red_spot': '/mnt/disk1/project/public/SMhospital/Database/1_ongoing/sm_core/total/hemorrhagic/red_spot', 'angioectasia': '/mnt/disk1/project/public/SMhospital/Database/1_ongoing/sm_core/total/hemorrhagic/angioectasia', 'active_bleeding': '/mnt/disk1/project/public/SMhospital/Database/1_ongoing/sm_core/total/hemorrhagic/active_bleeding'}}


In [5]:
list_data_aug = {}
for i in cls:
    list_data_aug[i] = {}
    for j in lesion[i]:
        list_data_aug[i][j] = os.listdir(path_folder[i][j])

print(list_data_aug)

{'negative': {'negative': ['r3_EC___02-39-07___2033654.jpg', 'f_r2_jb___04-53-21___2052285.jpg', 'r2_DJ___03-40-43___1026492.jpg', 'JH___04-10-09___1030024.jpg', 'CW___02-48-46___1020256.jpg', 'r2_DJ___03-21-28___1024181.jpg', 'f_r1_JY___02-02-16___1014675.jpg', 'r1_MP___04-53-50___1035269.jpg', 'f_r2_GN___03-50-55___1027711.jpg', 'EC___02-39-23___2033706.jpg', 'r3_jb___04-53-21___2052285.jpg', 'f_r2_GN___03-51-01___1027722.jpg', 'r2_EC___02-39-09___2033660.jpg', 'f_r2_CGB___06-31-10___1046950.jpg', 'JG___03-55-23___1028253.jpg', 'r3_jb___05-06-54___2053912.jpg', 'f_JS___03-59-38___1028727.jpg', 'r1_ML___04-06-04___1029535.jpg', 'JS___04-56-49___1035626.jpg', 'f_r3_JG___03-55-41___1028290.jpg', 'f_r3_HS___01-11-54___1008631.jpg', 'f_r2_JG___03-55-42___1028292.jpg', 'r1_EC___02-39-20___2033694.jpg', 'f_r3_DO___05-30-35___1039680.jpg', 'r3_AJ___03-13-59___1023283.jpg', 'f_r1_PI___04-16-40___2043214.jpg', 'r1_JY___02-03-06___1014775.jpg', 'r1_jb___03-47-44___2040894.jpg', 'f_r1_AJ___02-09

In [7]:
# dir_data = '/mnt/disk1/project/public/SMhospital/Database/1_ongoing/sm/total/'

# data_x = {'negative':{'negative': []},
#          'hemorrhagic':{'red_spot':[],
#                        'angioectasia':[],
#                        'active_bleeding':[]},
#          'depressed':{'erosion':[],
#                      'ulcer':[],
#                      'stricture':[]},
#          'protruded':{'ampulla_of_vater':[],
#                      'lymphoid_follicles':[],
#                      'small_bowel_tumor':[]}}
data_x = {'negative':{'negative': []},
         'hemorrhagic':{'red_spot':[],
                       'angioectasia':[],
                       'active_bleeding':[]}}

for i in cls:
    for j in lesion[i]:
        for k in list_data_aug[i][j]:
            temp = cv2.imread(dir_data + i + '/' + j + '/' + k)
            data_x[i][j].append([k, temp])

In [8]:
def sep(data_x):
    num_data = len(data_x)
    num_train_data = int(num_data*0.8)

    train_idx = set(np.random.choice(num_data, num_train_data, replace=False))
    test_idx = set(np.arange(len(data_x))) - train_idx
        
    train_x = [data_x[i] for i in train_idx]
    test_x = [data_x[i] for i in test_idx]
        
    return (train_x, test_x)

In [9]:
# train_data_x = {'negative':{'negative': []},
#                 'hemorrhagic':{'red_spot':[],
#                                'angioectasia':[],
#                                'active_bleeding':[]},
#                 'depressed':{'erosion':[],
#                              'ulcer':[],
#                              'stricture':[]},
#                 'protruded':{'ampulla_of_vater':[],
#                              'lymphoid_follicles':[],
#                              'small_bowel_tumor':[]}}
# test_data_x = {'negative':{'negative': []},
#                 'hemorrhagic':{'red_spot':[],
#                                'angioectasia':[],
#                                'active_bleeding':[]},
#                 'depressed':{'erosion':[],
#                              'ulcer':[],
#                              'stricture':[]},
#                 'protruded':{'ampulla_of_vater':[],
#                              'lymphoid_follicles':[],
#                              'small_bowel_tumor':[]}}
train_data_x = {'negative':{'negative': []},
                'hemorrhagic':{'red_spot':[],
                               'angioectasia':[],
                               'active_bleeding':[]}}
test_data_x = {'negative':{'negative': []},
                'hemorrhagic':{'red_spot':[],
                               'angioectasia':[],
                               'active_bleeding':[]}}

for i in cls:
    for j in lesion[i]:
        train_data_x[i][j], test_data_x[i][j] = sep(data_x[i][j])

In [12]:
train_dir_data = '/mnt/disk1/project/public/SMhospital/Database/1_ongoing/sm_core/train/'

for i in cls:
    for j in lesion[i]:
        for k in train_data_x[i][j]:
            cv2.imwrite(train_dir_data + '{0}/{1}/{2}'.format(i,j,k[0]), k[1])

In [13]:
test_dir_data = '/mnt/disk1/project/public/SMhospital/Database/1_ongoing/sm_core/test/'

for i in cls:
    for j in lesion[i]:
        for k in test_data_x[i][j]:
            cv2.imwrite(test_dir_data + '{0}/{1}/{2}'.format(i,j,k[0]), k[1])